In [6]:
import logging
import time
import os
import math
import ray
import random
import tqdm
from typing import Dict, Tuple, List
from random import randint
import numpy as np

In [7]:
if ray.is_initialized:
    ray.shutdown()
ray.init(logging_level=logging.ERROR)

Python version:,3.8.13
Ray version:,2.3.0
Dashboard:,http://127.0.0.1:8267


In [8]:
class ActorCls:
    def __init__(self, name: str):
        self.name = name
        self.method_calls = {"method": 0}

    def method(self, **args) -> None:
        # Overwrite this method in the subclass
        pass

    def get_all_method_calls(self) -> Tuple[str, Dict[str, int]]:
        return self.get_name(), self.method_calls
    
    def get_name(self) -> str:
        return self.name

In [9]:
@ray.remote
class ActorClsOne(ActorCls):
    
    def __init__(self, name: str):
        super().__init__(name)
        
    def method(self, **args) -> None:
        # do something with kwargs here
        time.sleep(args["timeout"])
        
        # update the respective counter
        self.method_calls["method"] += 1

In [10]:
@ray.remote
class ActorClsTwo(ActorCls):
    
    def __init__(self, name: str):
         super().__init__(name)
        
    def method(self, **args) -> None:
        # do something with kwargs here
        time.sleep(args["timeout"])
        
        # update the respective counter
        self.method_calls["method"] += 1

In [11]:
@ray.remote
class ActorClsThree(ActorCls):
    
    def __init__(self, name: str):
         super().__init__(name)
        
    def method(self, **args) -> None:
        # do something with kwargs here
        time.sleep(args["timeout"])
        
        # update the respective counter
        self.method_calls["method"] += 1

In [12]:
actor_one = ActorClsOne.remote("ActorClsOne")
actor_two = ActorClsTwo.remote("ActorClsTwo")
actor_three = ActorClsTwo.remote("ActorClsThree")

In [14]:
# A list of Actor classes
CALLERS_NAMES = ["ActorClsOne", "ActorClsTwo", "ActorClsThree"]

# A dictionary of Actor instances
CALLERS_CLS_DICT = {"ActorClsOne": actor_one, 
                    "ActorClsTwo": actor_two,
                    "ActorClsThree": actor_three}

In [15]:
count_dict = {"ActorClsOne": 0, "ActorClsTwo": 0, "ActorClsThree": 0}
for _ in range(len(CALLERS_NAMES)): 
    for _ in range(15):
        name = random.choice(CALLERS_NAMES)
        count_dict[name] += 1 
        CALLERS_CLS_DICT[name].method.remote(timeout=1, store="mongo_db") if name == "ActorClsOne" else CALLERS_CLS_DICT[name].method.remote(timeout=1.5, store="delta")
        
    print(f"State of counts in this execution: {count_dict}")
    time.sleep(0.5)

State of counts in this execution: {'ActorClsOne': 3, 'ActorClsTwo': 8, 'ActorClsThree': 4}
State of counts in this execution: {'ActorClsOne': 6, 'ActorClsTwo': 14, 'ActorClsThree': 10}
State of counts in this execution: {'ActorClsOne': 10, 'ActorClsTwo': 17, 'ActorClsThree': 18}


In [16]:
print(ray.get([CALLERS_CLS_DICT[name].get_all_method_calls.remote() for name in CALLERS_NAMES]))

[('ActorClsOne', {'method': 10}), ('ActorClsTwo', {'method': 17}), ('ActorClsThree', {'method': 18})]


In [17]:
ray.shutdown()